In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

In [ ]:
data_dir='data'


In [ ]:
from sklearn.preprocessing import scale, minmax_scale
from sklearn.metrics import root_mean_squared_error, ndcg_score
def calc_test(true_scores, pred_scores, k=10):
    rho, _ = stats.spearmanr(true_scores, pred_scores)

    # RMSE
    rmse = root_mean_squared_error(true_scores, pred_scores)

    # NDCG@k
    std_tgts = minmax_scale([true_scores], (0, 5), axis=1)
    ndcg_val = ndcg_score(std_tgts,[pred_scores], k=k)

    result ={
        'spearman': rho,
        # 'rmse': rmse,
        'ndcg': ndcg_val
    }
    return result

In [ ]:
import pandas as pd
from pygmo import hypervolume

def greedy_hypervolume_subset(points, n, ref_point):
    selected = []
    remaining = list(range(len(points)))

    for _ in tqdm(range(n)):
        max_hv = -float('inf')
        best_idx = None

        for idx in remaining:
            # 現在の選択 + 候補点のHypervolume計算
            current_points = points[selected + [idx]]
            hv = hypervolume(current_points)
            current_hv = hv.compute(ref_point)

            if current_hv > max_hv:
                max_hv = current_hv
                best_idx = idx

        if best_idx is not None:
            selected.append(best_idx)
            remaining.remove(best_idx)

    return selected, max_hv

def normalize_score(score):
    return (score-score.quantile(0.05))/(score.quantile(0.95)-score.quantile(0.05)+1e-10)


In [ ]:
import numpy as np
from typing import Union, Tuple

def calculate_mean_similarity(latent_matrix: np.ndarray):
    
    # 入力チェック
    if not isinstance(latent_matrix, np.ndarray):
        raise TypeError("latent_matrix must be numpy.ndarray")
    
    if len(latent_matrix.shape) != 2:
        raise ValueError("latent_matrix must be 2-dimensional")
        
    N, H = latent_matrix.shape
    
    if N < 2:
        raise ValueError("Number of samples must be greater than 1")
    
    # 各ベクトルのノルムを計算
    norms = np.linalg.norm(latent_matrix, axis=1, keepdims=True)
    # ゼロ除算を防ぐ
    norms = np.where(norms == 0, 1e-8, norms)
    
    # 正規化された行列を計算
    normalized_matrix = latent_matrix / norms
    
    # コサイン類似度行列を計算
    similarity_matrix = np.dot(normalized_matrix, normalized_matrix.T)
    # 対角要素を0にする（自己との類似度は除外）
    np.fill_diagonal(similarity_matrix, 0)
    
    # 平均コサイン類似度を計算
    mean_similarity = similarity_matrix.sum() / (N * (N-1))
        
    return mean_similarity


In [ ]:
import logomaker
def draw_logo(seqs, ax=None):
    if isinstance(seqs, str):
        seqs = [seqs]
    counts_matrix = logomaker.alignment_to_matrix(seqs)
    
    logo = logomaker.Logo(counts_matrix,
            shade_below=.5,
            fade_below=.5,
            color_scheme='NajafabadiEtAl2017',
            ax=ax
        )
    logo.ax.spines['right'].set_visible(False)
    logo.ax.spines['top'].set_visible(False)
    logo.ax.spines['bottom'].set_visible(False)
    logo.ax.spines['left'].set_visible(False)
    # logo.ax.set_xticks(np.arange(length))
    logo.ax.set_yticks([])


In [ ]:
h3_dict = {
    "4D5_HER2_fitness_1N8Z":   "SRWGGDGFYAMDY",
    "5A12_Ang2_fitness_4ZFG":  "ARFVFFLPYAMDY",
    "5A12_VEGF_fitness_4ZFF":  "ARFVFFLPYAMDY",
}

In [ ]:
def get_ddg(path):
    results_df = pd.read_csv(path)
    
    ddg_scores = (results_df[results_df["scored_state"]=="ddG"]
                     .groupby("case_name")["total_score"]
                     .min()
                     .sort_index())
    return ddg_scores

In [ ]:
flex_ddg_dfs={}
sampled_seq_dfs = {}
flex_ddg_df_alls = {}
for target in targets:
    for mode in ["bias", "unbias"]:
        flex_ddg_df = pd.read_csv(f"flexddgs/{target}/{mode}/outputs-results.csv")
        flex_ddg_df = flex_ddg_df[flex_ddg_df["scored_state"]=="ddG"].groupby("case_name")["total_score"].min().sort_index()
        flex_ddg_dfs[target+"_"+mode]=flex_ddg_df
        sampled_seq_dfs[target+"_"+mode]=pd.read_csv(f"flexddgs/{target}/{mode}/sampled_mutations.csv", index_col=0)
        
test_dfs = {target: pd.read_csv(f"flexddgs/{target}/bias/sampled_mutations.csv") for target in targets}
for target in test_dfs:
    test_dfs[target]["DMS_score"] = - flex_ddg_dfs[target+"_bias"].values

In [ ]:
import yaml

In [ ]:
jobdf = pd.read_csv("jobs/job.csv")
import os
cycles = {}
dfs = []
configs = []
ref_points = []
for confpath in jobdf["CONFIG"]:
    with open(confpath) as f:
        data = yaml.safe_load(f)
    target=data["data_dir"].split("/")[1]
    model_type = data["data_dir"].split("/")[2]
    exp=data["data_dir"].split("/")[3]
    df = pd.read_csv(os.path.join(data_dir, "..", data["data_dir"], "9", "train_data", "training_data.csv"))
    df["target"]=target
    df["model_type"]=model_type
    df["mutations"] = df["mutations"].fillna("")
    # df["mutations_wt"] = df["mutseq"].apply(lambda x: mutseq_to_mutstr(x, h3_dict["4D5_HER2_fitness_1N8Z"], "B", offset=0))
    df["exp"]=exp
    df["flxddg"] = -df["DMS_score"]
    score_cols = ["flxddg_std", "ablang2_perplexity_std", "IP_seq_std"]

    # hv
    df["flxddg_std"] = normalize_score(df["flxddg"])
    df["ablang2_perplexity_std"] = normalize_score(df["ablang2_perplexity"])
    df["IP_seq_std"] = normalize_score(-df["IP_seq"])
    
    df["flxddg_std"]*=2
    ref_points
    ref_point = []
    for col in score_cols:
        ref_point.append(df[col].quantile(0.95))
    ref_points.append(ref_point)
    # df["hv_score"] = df[score_cols].sum(axis=1)
    
    df["#Mutation"]=df["mutations"].apply(lambda x: len(x.split(",")) if x !="" else 0)
    dfs.append(df)
    configs.append({
        "target":target,
        "MAXCYCLE":10,
        "model_type": model_type,
        "exp":exp,
        "data_dir": data["data_dir"]
        
    })
    ddgs_list = []
    for cycle in range(10):
        ddgs = get_ddg(os.path.join(data_dir, "..", data["data_dir"], str(cycle), "flex_ddG", "outputs-results.csv"))
        ddgs = ddgs.reset_index()
        ddgs["cycle"]=cycle
        ddgs_list.append(ddgs)


In [ ]:
import yaml
from tqdm import tqdm

N=40

all_df_merges=[]
top_df_merges=[]
hv_df_merges=[]
for i in range(len(dfs)):
    target=configs[i]["target"]
    exp=configs[i]["exp"]
    df = dfs[i]
    CYCLE=configs[i]["MAXCYCLE"]
    top_dfs = {cycle+1: df[df["cycle"]<=cycle].sort_values("DMS_score", ascending=False).head(N)
               for cycle in range(CYCLE)}
    all_dfs = {cycle+1: df[df["cycle"]<=cycle] for cycle in range(CYCLE)}
    df_c = df.copy()
    for i in range(3):
        df_c = df_c[df_c[score_cols[i]] <= ref_point[i]]


    hv_dfs = {}
    for cycle in range(CYCLE):
        cycle_df = df_c[df_c["cycle"]<=cycle]
        selected_indices, _ = greedy_hypervolume_subset(cycle_df[score_cols].values, N, ref_point)
        hv_dfs[cycle+1] = cycle_df.iloc[selected_indices]

    top_df_merge = pd.concat(top_dfs)
    all_df_merge = pd.concat(all_dfs)
    hv_df_merge = pd.concat(hv_dfs)
    top_df_merge.index.names=["CYCLE", "index"]
    all_df_merge.index.names=["CYCLE", "index"]
    hv_df_merge.index.names=["CYCLE", "index"]
    top_df_merge = top_df_merge.reset_index()
    all_df_merge = all_df_merge.reset_index()
    hv_df_merge = hv_df_merge.reset_index()
    top_df_merges.append(top_df_merge)
    all_df_merges.append(all_df_merge)
    hv_df_merges.append(hv_df_merge)
top_df_merge_cat = pd.concat(top_df_merges)
all_df_merge_cat = pd.concat(all_df_merges)
hv_df_merge_cat = pd.concat(hv_df_merges)


In [ ]:
all_divs = []
top_divs = []
hv_divs = []
for i in range(len(dfs)):
    alldf = all_df_merges[i]
    topdf = top_df_merges[i]
    hvdf = hv_df_merges[i]

    conf=configs[i]
    input_dir = os.path.join(data_dir, conf["target"], conf["model_type"], conf["exp"], "9", "train_data")
    emb = np.load(os.path.join(input_dir, "embedding.npy"))
    # emb = np.load(os.path.join(input_dir, "embedding_umap5.npy"))
    
    # Calculate diversity metrics for all sequences
    divs = {cycle: 1-calculate_mean_similarity(emb[alldf[(alldf["CYCLE"]==cycle)]["index"].values]) for cycle in range(1,11)}
    mean_muts = pd.Series({cycle: alldf[alldf["CYCLE"]==cycle]["#Mutation"].mean() for cycle in range(1,11)})
    med_muts = pd.Series({cycle: alldf[alldf["CYCLE"]==cycle]["#Mutation"].median() for cycle in range(1,11)})
    divs = pd.Series(divs)
    divs.index.name="CYCLE"
    divs.name="Diversity"
    divs = divs.reset_index()
    divs["mean_mutation_num"] = mean_muts.values
    divs["median_mutation_num"] = med_muts.values
    divs["target"]=conf["target"]
    divs["model_type"]=conf["model_type"]
    divs["exp"]=conf["exp"]
    all_divs.append(divs)

    # Calculate diversity metrics for top sequences
    top_div = {cycle: 1-calculate_mean_similarity(emb[topdf[(topdf["CYCLE"]==cycle)]["index"].values]) for cycle in range(1,11)}
    top_mean_muts = pd.Series({cycle: topdf[topdf["CYCLE"]==cycle]["#Mutation"].mean() for cycle in range(1,11)})
    top_med_muts = pd.Series({cycle: topdf[topdf["CYCLE"]==cycle]["#Mutation"].median() for cycle in range(1,11)})
    top_div = pd.Series(top_div)
    top_div.index.name="CYCLE"
    top_div.name="Diversity"
    top_div = top_div.reset_index()
    top_div["mean_mutation_num"] = top_mean_muts.values
    top_div["median_mutation_num"] = top_med_muts.values
    top_div["target"]=conf["target"]
    top_div["model_type"]=conf["model_type"]
    top_div["exp"]=conf["exp"]
    top_divs.append(top_div)

    # Calculate diversity metrics for hypervolume sequences
    hv_div = {cycle: 1-calculate_mean_similarity(emb[hvdf[(hvdf["CYCLE"]==cycle)]["index"].values]) for cycle in range(1,11)}
    hv_mean_muts = pd.Series({cycle: hvdf[hvdf["CYCLE"]==cycle]["#Mutation"].mean() for cycle in range(1,11)})
    hv_med_muts = pd.Series({cycle: hvdf[hvdf["CYCLE"]==cycle]["#Mutation"].median() for cycle in range(1,11)})
    hv_div = pd.Series(hv_div)
    hv_div.index.name="CYCLE"
    hv_div.name="Diversity"
    hv_div = hv_div.reset_index()
    hv_div["mean_mutation_num"] = hv_mean_muts.values
    hv_div["median_mutation_num"] = hv_med_muts.values
    hv_div["target"]=conf["target"]
    hv_div["model_type"]=conf["model_type"]
    hv_div["exp"]=conf["exp"]
    hv_divs.append(hv_div)

all_divs_cat = pd.concat(all_divs, ignore_index=True)
top_divs_cat = pd.concat(top_divs, ignore_index=True)
hv_divs_cat = pd.concat(hv_divs, ignore_index=True)


In [ ]:
all_test_scores=[]
for conf in configs:
    target = conf["target"]
    if target not in targets:
        continue
    for cycle in range(10):
        input_dir = os.path.join(data_dir, conf["target"], conf["model_type"], conf["exp"], str(cycle), "train_data")
        test_pred = np.load(os.path.join(input_dir, "test_inference_bias.npy"))
        test_df = test_dfs[target]
        test_df_ = test_df.copy()
        test_df_["Pred"] = test_pred
        all_test_scores.append({
            **calc_test(test_df_["DMS_score"], test_df_["Pred"]),
            "CYCLE": cycle+1,
            "target": conf["target"],
            "model_type": conf["model_type"],
            "exp": conf["exp"],
        })
all_test_scores_cat = pd.DataFrame(all_test_scores)
all_test_scores_cat["spearman"] = all_test_scores_cat["spearman"].fillna(0)

In [ ]:
top_df_merge_cat_sel.to_csv("results/flexddg_offline/top_results.csv",index=False)
all_test_scores_cat_sel.to_csv("results/flexddg_offline/all_results_test.csv",index=False)
all_df_merge_cat_sel.to_csv("results/flexddg_offline/all_results.csv",index=False)